# ANÁLISIS DE BALANCE DE FLUJOS (FBA) MEDIANTE COBRA 

El Análisis de Balance de Flujo (FBA) es un método de optimización matemática que predice los flujos metabólicos en un organismo usando modelos basados en redes metabólicas. Se basa en la conservación de masa, las restricciones de flujos y la suposición de estado estacionario (los metabolitos no se acumulan).
Esencialmente consiste en la utilización de un modelo a escala del genoma (GEM, genome-scale metabolic model) el cual contiene muchos genes de un organismo con sus reacciones estequiométricamente balanceadas así como los metabolitos involucrados. Estos modelos son utilizados para crear una matriz estequiómetrica que al multiplicarla por el vector de flujos metabólicos y suponiendo estado estacionario nos da un sistema de ecuaciones lineales a resolver. 

No obstante, como hay más reacciones que metabolitos en los modelos, la solución a la optimización que se desea realizar no tiene una única solución sino varias que se pueden graficar en un espacio de solución multidimensional. Para dar con soluciones aproximadas se añade al modelo los límtes de las reacciones (upper y lower bounds), los cuales dan los flujos máximos y mínimos de cada reacción basados en datos experimentales generalmente conseguidos por marcado con el radioisótopo carbono 13. Adicionalmente, estos límites son muy importantes porque indican que reacciones son reversibles, si están favorecidas hacia productos o reactivos y cuales son de importación o exportación de metabolitos entre citosol y espacio extracelular.

Dado que estos sistemas de ecuaciones consisten de centenas de ecuaciones y variables, se hace uso de librerías de solución por optimización con programación lineal como lo es el COBRA toolbox (COnstraint Based Reconstruction and Analysis) el cual se utiliza ampliamente en los lenguajes de programación MATLAB y Python usando a su vez archivos xml escritos en lenguaje SBML (Systems Biology Markup Language) para cargar los modelos a escala del genoma.

Ahora bien, veamos los fundamentos de cómo se usa COBRA, particularmente cobrapy (la versión de Python) para hacer FBA:

In [1]:
# se carga la librería cobra
import cobra
from cobra.io import load_model

Cargamos el modelo correspodiente, puede ser uno de los que vienen integrados en cobrapy o el modelo específico que se va a usar proveniente desde la ruta donde está ubicado

In [2]:
# "iJO1366" y "salmonella" también son argumentos válidos pues vienen por defecto en cobrapy
model = load_model("textbook")

Set parameter Username
Set parameter LicenseID to value 2695449
Academic license - for non-commercial use only - expires 2026-08-13


Podemos ver la cantidad de reacciones, genes y metabolitos que tiene un modelo; en este caso el modelo es del metabolismo central de E. coli 

In [3]:
print(len(model.reactions))
print(len(model.metabolites))
print(len(model.genes))

95
72
137


Podemos ver en qué consiste el modelo en sí:

In [4]:
model

Name,e_coli_core
Memory address,183fb4be190
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"cytosol, extracellular"


Como se puede observar, la función objetivo, es decir la que se desea maximizar o minimizar o en general optimizar es la biomasa por defecto. Además, la mayoría de estos modelos vienen compartimentalizados por ejemplo en citosol y extracelular para dar lugar a proteínas y reacciones de transporte e intercambio

Podemos llamar un metabolito, reacción o gen según su ID (el cual cambia según el modelo):

In [5]:
model.metabolites.get_by_id("glc__D_e")

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x183827acc90
Formula,C6H12O6
Compartment,e
In 2 reaction(s),"GLCpts, EX_glc__D_e"


In [6]:
model.reactions.get_by_id("PFK")

Reaction identifier,PFK
Name,phosphofructokinase
Memory address,0x1838384cd50
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP + D-Fructose 6-phosphate --> ADP + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [7]:
model.genes.get_by_id("b4025")

Gene identifier,b4025
Name,pgi
Memory address,0x183827e9050
Functional,True
In 1 reaction(s),PGI


Ya teniendo el modelo se pueden modificar los parámetros que uno quiera por ejemplo, los límites superiores e inferiores de los flujos de las reacciones, añadir o eliminar reacciones, genes o metabolitos o incluso hacer gene knock out:

In [8]:
# Especificar el ID del gen a inactivar; por ejemplo, 'b4025' que codifica para la glucosa-6-fosfato isomerasa
gene_id = 'b4025'
gene = model.genes.get_by_id(gene_id)

# Inactivar el gen estableciendo su 'knock_out' en True
gene.knock_out()

# Optimizar el modelo para evaluar el efecto del knockout en la tasa de crecimiento
solution = model.optimize()

# Imprimir el estado de la solución y la nueva tasa de crecimiento
print(f"Estado de la solución: {solution.status}")
print(f"Tasa de crecimiento después del knockout de {gene_id}: {solution.objective_value:.5f} h⁻¹")

Estado de la solución: optimal
Tasa de crecimiento después del knockout de b4025: 0.86316 h⁻¹


In [12]:
for reaction in model.reactions[:15]:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

ACALD blocked (bounds: (0, 0)), new growth rate 0.863160
ACALDt blocked (bounds: (0, 0)), new growth rate 0.863160
ACKr blocked (bounds: (0, 0)), new growth rate 0.863160
ACONTa blocked (bounds: (0, 0)), new growth rate 0.000000
ACONTb blocked (bounds: (0, 0)), new growth rate 0.000000
ACt2r blocked (bounds: (0, 0)), new growth rate 0.863160
ADK1 blocked (bounds: (0, 0)), new growth rate 0.863160
AKGDH blocked (bounds: (0, 0)), new growth rate 0.856500
AKGt2r blocked (bounds: (0, 0)), new growth rate 0.863160
ALCD2x blocked (bounds: (0, 0)), new growth rate 0.863160
ATPM blocked (bounds: (0, 0)), new growth rate 0.907741
ATPS4r blocked (bounds: (0, 0)), new growth rate 0.295782
Biomass_Ecoli_core blocked (bounds: (0, 0)), new growth rate 0.000000
CO2t blocked (bounds: (0, 0)), new growth rate 0.197025
CS blocked (bounds: (0, 0)), new growth rate 0.000000


Los modelos resueltos mediante FBA pueden analizarse más a fondo utilizando métodos de resumen, los cuales generan texto impreso para proporcionar una representación rápida del comportamiento del modelo. Al llamar al método de resumen en todo el modelo, se muestra información sobre el comportamiento de entrada y salida del modelo, junto con el valor optimizado de la función objetivo.

In [13]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.707,0,0.00%
o2_e,EX_o2_e,22.27,0,0.00%
pi_e,EX_pi_e,3.175,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-23.27,1,100.00%
h2o_e,EX_h2o_e,-29.56,0,0.00%
h_e,EX_h_e,-17.31,0,0.00%


Además de FBA, también se pueden ejecutar otros tipos de análisis tales como FBA parsimonioso (pFBA), Análisis de Variabilidad de Flujos (FVA) o FBA dinámico, es decir en estado transitorio, no estacionario (dFBA)